### Validate our Environment

**Check Python version. This notebook is implemented for Python 3.5.x. Not all cells may work in other versions of Python.**

In [ ]:
import platform
print(platform.python_version())

# Predicting Customer Churn in Telco

In this notebook you will learn how to build a predictive model with Spark machine learning API (SparkML) and deploy it for scoring in Machine Learning (ML). 

This notebook walks you through these steps:
- Build a model with SparkML API
- Save the model in the ML repository
- Create a Deployment in ML (via UI)
- Test the model (via UI)
- Test the model (via REST API)

## Step 1: Review Use Case

The analytics use case implemented in this notebook is telco churn. While it's a simple use case, it implements all steps from the CRISP-DM methodolody, which is the recommended best practice for implementing predictive analytics. 
![CRISP-DM](https://raw.githubusercontent.com/yfphoon/dsx_demo/master/crisp_dm.png)

The analytics process starts with defining the business problem and identifying the data that can be used to solve the problem. For Telco churn, we use demographic and historical transaction data. We also know which customers have churned, which is the critical information for building predictive models. In the next step, we use visual APIs for data understanding and complete some data preparation tasks. In a typical analytics project data preparation will include more steps (for example, formatting data or deriving new variables). 

Once the data is ready, we can build a predictive model. In our example we are using the SparkML Random Forrest classification model. Classification is a statistical technique which assigns a "class" to each customer record (for our use case "churn" or "no churn"). Classification models use historical data to come up with the logic to predict "class", this process is called model training. After the model is created, it's usually evaluated using another data set. 

Finally, if the model's accuracy meets the expectations, it can be deployed for scoring. Scoring is the process of applying the model to a new set of data. For example, when we receive new transactional data, we can score the customer for the risk of churn.  

We also developed a sample Python Flask application to illustrate deployment: http://predictcustomerchurn.mybluemix.net/. This application implements the REST client call to the model.

### Prerequisite:  Upgrade Pixiedust and dependencies

## **<span style="color:red"> Action Required </span>** 
** Restart Kernel after Pixiedust Upgrade **

In [ ]:
!pip install --user --upgrade pixiedust py4j jinja2

![RESTART KERNEL](https://raw.githubusercontent.com/krondor/ICP4D-/master/pics/restart_kernel.png)

### Working with Notebooks

If you are new to Notebooks, here's a quick overview of how to work in this environment.

1. The notebook has 2 types of cells - markdown (text) and code. 
2. Each cell with code can be executed independently or together (see options under the Cell menu). When working in this notebook, we will be running one cell at a time because we need to make code changes to some of the cells.
3. To run the cell, position cursor in the code cell and click the Run (arrow) icon. The cell is running when you see the * next to it. Some cells have printable output.
4. Work through this notebook by reading the instructions and executing code cell by cell. Some cells will require modifications before you run them. 

## Step 2: Load data 
** In this section, you will learn how to merge two datasets from different sources. The customer data is from a remote conncection and the churn data is from your local project. **
### i. Load the remote connection -  ENHANCED CUSTOMER TABLE: </br>
Before running the following steps, click the blank cell below. Then click the '1010' button on the top right on the top right to import dataset into notebook. Choose the **'Remote'** option and click the prepared merged data in that cell **CUSTOMER**. When importing the data, you can choose **'insert Spark DataFrame in Python'**. You could keep the name by default, or could rename it to '**df1**'. But make sure that the name is consistent in the following steps.

<img src="https://raw.githubusercontent.com/Rui425/ICP4D-/master/pics/InsertRemoteData.png" width="500">

You could keep the name by default, or could rename it to '**df1**'. But make sure that the name is consistent in the following steps.

In [ ]:
# Import the local churn file 
import os
from pyspark.sql import SQLContext

# Add asset from file system
df1 = SQLContext(sc).read.csv(os.environ['DSX_PROJECT_DIR']+'/datasets/customer.csv', header='true', inferSchema = 'true')
df1.show(5)





### ii .Load the local project data - Churn.csv:

In [ ]:
# Import the local churn file 
import os
from pyspark.sql import SQLContext
# Add asset from file system
churn = SQLContext(sc).read.csv(os.environ['DSX_PROJECT_DIR']+'/datasets/churn.csv', header='true', inferSchema = 'true')
churn.show(5)

If the first step ran successfully (you saw the output), then continue reviewing the notebook and running each code cell step by step. Note that not every cell has a visual output. The cell is still running if you see a * in the brackets next to the cell. 

If the first step didn't finish successfully, check with the instructor. 

## Step 3: Merge Files
While merging files, making sure that you are using the remote connected data - customer(df1 or other version) and churn data. The name of the enhanced customer data table may be different. You should make sure that the name is consistent with the name when importing. 

In [ ]:
data = df1.join(churn,df1['ID']==churn['ID']).select(df1['*'],churn['CHURN'])
data.show(5)

## Step 4: Data Prep
A typical data scientist workflow may involve lots of data prep.  This is a simple example of this process, we can rename columns, define data types, or otherwise manipulate data elements.  


In [ ]:
data = data.withColumnRenamed("Est Income", "ESTINCOME").withColumnRenamed("Car Owner","CAROWNER").withColumnRenamed("Gender","GENDER")\
.withColumnRenamed("Status","STATUS").withColumnRenamed("Children","CHILDREN").withColumnRenamed("Age","AGE")\
.withColumnRenamed("LongDistance","LONGDISTANCE").withColumnRenamed("International","INTERNATIONAL")\
.withColumnRenamed("Local","LOCAL").withColumnRenamed("Dropped","DROPPED").withColumnRenamed("Paymethod","PAYMETHOD")\
.withColumnRenamed("LocalBilltype","LOCALBILLTYPE").withColumnRenamed("LongDistanceBilltype","LONGDISTANCEBILLTYPE")\
.withColumnRenamed("Usage","USAGE").withColumnRenamed("RatePlan","RATEPLAN")
## Schema Transformation Example (AGE to Integer)
# df1 = df1.withColumn('AGE', df1['AGE'].cast('int'))
data.show(1)

## Step 5: Data understanding

Data preparation and data understanding are the most time-consuming tasks in the data mining process. The data scientist needs to review and evaluate the quality of data before modeling.

Visualization is one of the ways to reivew data.

The Brunel Visualization Language is a highly succinct and novel language that defines interactive data visualizations based on tabular data. The language is well suited for both data scientists and business users. 
More information about Brunel Visualization: https://github.com/Brunel-Visualization/Brunel/wiki

Try Brunel visualization here: http://brunel.mybluemix.net/gallery_app/renderer

In [ ]:
import brunel
df = data.toPandas()
%brunel data('df') bar x(CHURN) y(ESTINCOME) mean(ESTINCOME) color(LOCALBILLTYPE) stack tooltip(ESTINCOME) | x(LONGDISTANCE) y(USAGE) point color(PAYMETHOD) tooltip(LONGDISTANCE, USAGE) :: width=1100, height=400 


**PixieDust** is a Python Helper library for Spark IPython Notebooks. One of it's main features are visualizations. You'll notice that unlike other APIs which produce just output, PixieDust creates an **interactive UI** in which you can explore data.

More information about PixieDust: https://github.com/ibm-cds-labs/pixiedust?cm_mc_uid=78151411419314871783930&cm_mc_sid_50200000=1487962969

In [ ]:
from pixiedust.display import *
display(data)

Pixiedust can also explore Spark Dataframes.  You may need to customize the following cell for your Spark Dataframe declaration.

In [ ]:
display(df1)

## Step 6: Build the Spark pipeline and the Random Forest model
"Pipeline" is an API in SparkML that's used for building models.
Additional information on SparkML: https://spark.apache.org/docs/2.0.2/ml-guide.html

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer, IndexToString
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

# Prepare string variables so that they can be used by the decision tree algorithm
# StringIndexer encodes a string column of labels to a column of label indices
SI1 = StringIndexer(inputCol='GENDER', outputCol='GenderEncoded')
SI2 = StringIndexer(inputCol='STATUS',outputCol='StatusEncoded')
SI3 = StringIndexer(inputCol='CAROWNER',outputCol='CarOwnerEncoded')
SI4 = StringIndexer(inputCol='PAYMETHOD',outputCol='PaymethodEncoded')
SI5 = StringIndexer(inputCol='LOCALBILLTYPE',outputCol='LocalBilltypeEncoded')
SI6 = StringIndexer(inputCol='LONGDISTANCEBILLTYPE',outputCol='LongDistanceBilltypeEncoded')
labelIndexer = StringIndexer(inputCol='CHURN', outputCol='label').fit(data)

# Pipelines API requires that input variables are passed in  a vector
assembler = VectorAssembler(inputCols=["GenderEncoded", "StatusEncoded", "CarOwnerEncoded", "PaymethodEncoded", "LocalBilltypeEncoded", \
                                       "LongDistanceBilltypeEncoded", "CHILDREN", "ESTINCOME", "AGE", "LONGDISTANCE", "INTERNATIONAL", "LOCAL",\
                                      "DROPPED","USAGE","RATEPLAN"], outputCol="features")

In [ ]:
# instantiate the algorithm, take the default settings
rf=RandomForestClassifier(labelCol="label", featuresCol="features")

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)

pipeline = Pipeline(stages=[SI1,SI2,SI3,SI4,SI5,SI6,labelIndexer, assembler, rf, labelConverter])

In [ ]:
# Split data into train and test datasets
train, test = data.randomSplit([0.7,0.3], seed=6)
train.cache()
test.cache()

In [ ]:
# Build models
model = pipeline.fit(train)

In [ ]:
model.transform(test)

## Step 7: Score the test data set

In [ ]:
results = model.transform(test)
results=results.select(results["ID"],results["CHURN"],results["label"],results["predictedLabel"],results["prediction"],results["probability"])
results.toPandas().head(6)

## Step 8: Model Evaluation 

In [ ]:
print('Precision model = {:.2f}.'.format(results.filter(results.label == results.prediction).count() / float(results.count())))

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label", metricName="areaUnderROC")
print('Area under ROC curve = {:.2f}.'.format(evaluator.evaluate(results)))

We have finished building and testing a predictive model. The next step is to deploy it for real time scoring. 

## Step 9: Save Model in ML repository

In [ ]:
from dsx_ml.ml import save

model_name = "Telco_Churn_ML_model"
save(name = model_name,
     model = model,
     algorithm_type = 'Classification',
     test_data = test)

In [ ]:
# Write the test data to a .csv so that we can later use it for Evaluation
writeCSV=test.toPandas()
writeCSV.to_csv('../datasets/TelcoModelEval.csv', sep=',', index=False)

# Let's also write out our merged dataset for later use as well
df.to_csv('../datasets/merged_customers.csv', sep=',', index=False)

## Step 10: Test Saved Model with Test UI

1. Save the notebook and switch to the **Models** tab of the project (**hint**: right click the project name link, DSX_Lo, at the top, and open with another tab in your browser). 
2. 
3. Under **Models**, find and click into your saved model. 
4. Click the **Test** link to test the model. You can use the following data for testing: <br/>
`ID=99, Gender=M, Status=S, Children=0, Est Income=60000, Car Owner=Y, Age=34, LongDistance=68, International=50, Local=100, Dropped=0, Paymethod=CC, LocalBilltype=Budget, LongDistanceBilltype=Intnl_discount, Usage=334, RatePlan=3`

The results of the test is displayed as follows:<br/>
<img style="float: left;" src="https://github.com/yfphoon/dsx_local/blob/master/images/Test_Model.png?raw=true" alt="Test API" width=900 />

## Step 11:  Test model with a REST API call (Optional)

This step demonstrates an "internal REST API" call to test the model (for an unpublished model). Notice that we are using DSX variables for the model endpoint and token. See documentation for external REST call syntax. An exernal REST call will have a different end point and will require authentication. 

In [ ]:
json_payload = [{
    "PHASE":"Adult",
    "ID":999,
    "GENDER":"F",
    "STATUS":"M",
    "CHILDREN":2.0,
    "ESTINCOME":77551.100000,
    "CAROWNER":"Y",
    "AGE":33,
    "LONGDISTANCE":20.530000,
    "INTERNATIONAL":0.000000,
    "LOCAL":41.890000,
    "DROPPED":1.000000,
    "PAYMETHOD":"CC",
    "LOCALBILLTYPE":"Budget",
    "LONGDISTANCEBILLTYPE":"Standard",
    "USAGE":62.420000,
    "RATEPLAN":2.000000
}]

## **<span style="color:red"> Action Required </span>** 
Change the *scoring_endpoint* to the value that's shown as the *scoring_endpoint* afer running Save to ML repository function (see **Step 9**), for example *'scoring_endpoint': 'https://dsxl-api.ibm-private-cloud.svc.cluster.local/v3/project/score/Python27/spark-2.0/DSX_Local_Workshop_el/Telco_Churn_ML_model/1'*. 

In [ ]:
import requests, json
from pprint import pprint

scoring_endpoint = ''

header_online = {'Content-Type': 'application/json', 'Authorization':os.environ['DSX_TOKEN']}

response_scoring = requests.post(scoring_endpoint, json=json_payload, headers=header_online)


In [ ]:
prediction = response_scoring.json()['object']['output']['predictions'][0]
print ('Prediction = {}'.format(prediction))
probabilities = response_scoring.json()['object']['output']['probabilities'][0]
print ('Probabilities = {}'.format(probabilities))
if prediction == 'F':
    print('Prediction = False')
    print('Probability = {0:.2f}'.format(probabilities[0]*100))
elif prediction == 'T':
    print('Prediction = True')
    print('Probability = {0:.2f}%'.format(probabilities[1]*100))
else:
    print('Probability ERROR')

### Summary

You have finished working on this hands-on lab. In this notebook you created a model using SparkML API, deployed it in  Machine Learning service for online (real time) scoring and tested it using a test client. 


Created by **Sidney Phoon**, **Elena Lowery** and **Rui Fan**
<br/>
yfphoon@us.ibm.com<br/>
elowery@us.ibm.com<br/>
rui.fan@ibm.com
<br/><br/>
May29, 2018

## Optional:  Evaluate Alternate Model Performance

In the example above, we used a random forest algorithm to generate our predictive model.  Was Random Forest our best option for this dataset?  Let's take a look at a gradient boosted tree model against our test and training sets and see how things may differ...

In [ ]:
from pyspark.ml.classification import GBTClassifier

# instantiate the algorithm, take the default settings
gbt = GBTClassifier(labelCol="label", featuresCol="features")

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)

pipeline = Pipeline(stages=[SI1,SI2,SI3,SI4,SI5,SI6,labelIndexer, assembler, gbt, labelConverter])

# Build models
model = pipeline.fit(train)
model.transform(test)

# Results
results = model.transform(test)
results=results.select(results["ID"],results["CHURN"],results["label"],results["predictedLabel"],results["prediction"],results["probability"])
results.toPandas().head(6)

from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Precision Model
print('Precision model1 = {:.2f}.'.format(results.filter(results.label == results.prediction).count() / float(results.count())))

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label", metricName="areaUnderROC")
print('Area under ROC curve = {:.2f}.'.format(evaluator.evaluate(results)))

It appears that GBT may outperform RF, however we should consider overfitting and hyperparameters.  

In [ ]:
from dsx_ml.ml import save

model_name = "Telco_Churn_ML_model_GBT"
save(name = model_name,
     model = model,
     algorithm_type = 'Classification',
     test_data = test)